In [54]:
import numpy as np
# Problem 1: Recap of Today’s Notes
# Exercise 1, faulty range sensor
# Working through understanding
p_0_faulty = 0.015  # original prior faulty
p_0_not = 1 - p_0_faulty    # original prior not faulty

# Calculate normalization
p_less_faulty = 1 * p_0_faulty
p_less_not = 0.25 * p_0_not
normalization_1 = p_less_not + p_less_faulty

p_less_faulty_1 = 1 # Prob r < 1 given sensor faulty
p_less_not_1 = 0.25 # Prob r < 1 given sensor not faulty

p_faulty_less_1 = p_less_faulty_1 * p_0_faulty / normalization_1
p_not_less_1 = p_less_not_1 * p_0_not / normalization_1

# Function implementation
def bayes_filter(priors):
    likelihood = np.array([0.25, 1]) # Prob r < 1 given sensor not faulty, prob r < 1 given sensor faulty [not faulty, faulty]
    posterior_temp = likelihood * priors    # Numerator
    posterior = posterior_temp / np.sum(posterior_temp)
    return posterior

priors = np.array([p_0_not, p_0_faulty])

for i in range(10):
    priors = bayes_filter(priors)
    print(f"Timestep {i + 1}: {priors}")    # [prob. not faulty, prob. faulty]


Timestep 1: [0.94258373 0.05741627]
Timestep 2: [0.80408163 0.19591837]
Timestep 3: [0.50642674 0.49357326]
Timestep 4: [0.20414508 0.79585492]
Timestep 5: [0.06026308 0.93973692]
Timestep 6: [0.01577893 0.98422107]
Timestep 7: [0.00399198 0.99600802]
Timestep 8: [0.00100099 0.99899901]
Timestep 9: [2.50435720e-04 9.99749564e-01]
Timestep 10: [6.26206918e-05 9.99937379e-01]


In [ ]:
import numpy as np
# Problem 1: Recap of Today’s Notes
# Exercise 2, "whack-a-mole" problem

# Working through problem
priors1 = np.array([1, 0, 0])  # Rooms 1, 2, 3 at time step 1
T = np.array([[0, 0, 0], [0.5, 0.8, 0.3], [0.5, 0.2, 0.7]]).T
print(priors1 @ T)
priors2 = priors1 @ T
print(priors2 @ T)

# Using transition matrix for probability distribution at each timestep
priors = priors1
num_timesteps = 9
for i in range (num_timesteps):
    priors = priors @ T
    print(f"Timestep {i+2}: {priors}")

# The probability distribution converges to 0.6 for room 2 and 0.4 for room 3

[0.  0.5 0.5]
[0.   0.55 0.45]
Timestep 2: [0.  0.5 0.5]
Timestep 3: [0.   0.55 0.45]
Timestep 4: [0.    0.575 0.425]
Timestep 5: [0.     0.5875 0.4125]
Timestep 6: [0.      0.59375 0.40625]
Timestep 7: [0.       0.596875 0.403125]
Timestep 8: [0.        0.5984375 0.4015625]
Timestep 9: [0.         0.59921875 0.40078125]
Timestep 10: [0.         0.59960938 0.40039062]


In [ ]:
import numpy as np
# Version 2: Noisy measurements
T = np.array(
    [
        [0, 0, 0],
        [0.5, 0.8, 0.3],
        [0.5, 0.2, 0.7],
    ]
).T # Transition matrix THIS IS TRANSPOSED
M = np.array(
    [
        [0, 0.5, 0.5],
        [0, 0.9, 0.1],
        [0, 0.1, 0.9],
    ]
) # Measurement matrix

prior_0 = np.array([1, 0, 0])   # Initial state
zs = np.array([2, 3, 3, 2, 3] - np.ones(5), dtype=int)   # Observations, -1 for each obeservation for indexing measurement and transition matrices

initial_weight = prior_0 * M[:, zs[0]]

def forward(weights, observation):
    likelihood = M[:, observation]
    prior = (weights @ T)
    return likelihood * prior

# Calculate forward steps
weights_list = [0] * len(zs)
weights = initial_weight
weights_list[0] = weights
print(f"Forward Step for k=1: {weights}")

for idx in range(len(zs) - 1):
    weights = forward(weights, zs[idx+1])
    weights_list[idx+1] = weights
    print(f"Forward Step for k={idx+2}: {weights}")

print()
# Calculate Filtered Estimates (Normalization and apply to each forward step)
for idx in range(len(zs)):
    posterior = weights_list[idx] / sum(weights_list[idx])
    print(f"Filtered Estimate for k={idx+1}: {posterior}")

print()
# Calculate Backwards Step
beta_k_5 = np.array([1, 1, 1])
beta_k = beta_k_5
betas_list = [0] * len(zs)
betas_list[-1] = beta_k_5
print(f"Backwards Step for k=5: {beta_k_5}")

for idx in reversed(range(len(zs)-1)):
    beta_k = T @ (beta_k * M[zs[idx+1]])
    betas_list[idx] = beta_k
    print(f"Backwards Step for k={idx+1}: {beta_k}")

print()
# Calculate Smoothing
for idx in range(len(zs)):
    numerator = weights_list[idx] * betas_list[idx]
    print(f"Smoothed Value for k={idx+1}: {(numerator / sum(numerator))}")

Forward Step for k=1: [0.5 0.  0. ]
Forward Step for k=2: [0.    0.025 0.225]
Forward Step for k=3: [0.      0.00875 0.14625]
Forward Step for k=4: [0.        0.0457875 0.0104125]
Forward Step for k=5: [0.         0.00397538 0.01480163]

Filtered Estimate for k=1: [1. 0. 0.]
Filtered Estimate for k=2: [0.  0.1 0.9]
Filtered Estimate for k=3: [0.         0.05645161 0.94354839]
Filtered Estimate for k=4: [0.        0.8147242 0.1852758]
Filtered Estimate for k=5: [0.         0.21171513 0.78828487]

Backwards Step for k=5: [1 1 1]
Backwards Step for k=4: [0.5  0.26 0.66]
Backwards Step for k=3: [0.15   0.2004 0.1164]
Backwards Step for k=2: [0.0624   0.036984 0.079344]
Backwards Step for k=1: [0.037554   0.01724064 0.05109624]

Smoothed Value for k=1: [1. 0. 0.]
Smoothed Value for k=2: [0.         0.04924109 0.95075891]
Smoothed Value for k=3: [0.         0.09338552 0.90661448]
Smoothed Value for k=4: [0.         0.63400703 0.36599297]
Smoothed Value for k=5: [0.         0.21171513 0.78828

In [ ]:
import numpy as np
# Problem 3: A Simple HVAC System
T = np.array(
    [
        [0.8, 0.2, 0],
        [0.4, 0.4, 0.2],
        [0.2, 0.6, 0.2],
    ]
) # Transition matrix 
M = np.array(
    [
        [0.6, 0.4, 0],
        [0.3, 0.7, 0],
        [0, 0, 1],
    ]
) # Measurement matrix

# Part A prediction
weather_0 = np.array([1, 0, 0])   # Sunny, cloudy, rainy
weather = weather_0

for day in range(4):
    weather = weather @ T

print(f"The probability of it being sunny, cloudy, or rainy on day 5 is {weather}, so it will likely still be sunny.\n")

# Part B
zs = [2, 1, 1, 2, 0]    # Observations, 0: sunny, 1: cloudy, 2: rainy
weather_day_1 = [0, 0, 1]

initial_weight = weather_day_1 * M[:, zs[0]]

def forward(weights, observation):
    likelihood = M[:, observation]
    prior = (weights @ T)
    return likelihood * prior

# Calculate forward steps
weights_list = [0] * len(zs)
weights = initial_weight
weights_list[0] = weights
print(f"Forward Step for k=1: {weights}")

for idx in range(len(zs) - 1):
    weights = forward(weights, zs[idx+1])
    weights_list[idx+1] = weights
    print(f"Forward Step for k={idx+2}: {weights}")

print()
# Calculate Filtered Estimates (Normalization and apply to each forward step)
for idx in range(len(zs)):
    posterior = weights_list[idx] / sum(weights_list[idx])
    print(f"Filtered Estimate for k={idx+1}: {posterior}")

# Calculate Backwards Step
beta_k_5 = np.array([1, 1, 1])
beta_k = beta_k_5
betas_list = [0] * len(zs)
betas_list[-1] = beta_k_5
print(f"\nBackwards Step for k=5: {beta_k_5}")

for idx in reversed(range(len(zs)-1)):
    beta_k = T @ (beta_k * M[zs[idx+1]])
    betas_list[idx] = beta_k
    print(f"Backwards Step for k={idx+1}: {beta_k}")

print()
# Calculate Smoothing
smoothed_values = [0] * len(zs)

for idx in range(len(zs)):
    numerator = weights_list[idx] * betas_list[idx]
    smoothed_values[idx] = numerator / sum(numerator)
    print(f"Smoothed Value for k={idx+1}: {(smoothed_values[idx])}")

print(f"The probability that the last day is actually sunny is {smoothed_values[-1][0]}.")


"""
Part C What was the most likely weather on each of days 2-4, using the observations from Part B?
The most likely weather on days 2-4 were cloudy, cloudy, rainy, respectively.
"""

"""
Part D Given what you’re observing about your state estimation capabilities, how would you go about evaluating your prototype HVAC system?
What caveats of your empirical performance would you need to communicate to a possible stakeholder?

To evaluate my prototype HVAC system I would compare the filtering and smoothing values of my system. If the trend seen in filtering held up after being smoothed I believe
with a decently accurate sensor, senses the correct state over 50%, then the HVAC system is performing well. For example in filtering on day 2 it was likely cloudy and on
day 3 it was rainy. The same can be seen after smoothing. However filtering in day 3 it's 10% more likely to be cloudy, but after smoothing it's guaranteed to be cloudy.

A caveat of the empirical testing to communicate to a possible stake holder is that a sunny day was only handled or detected at the end, so it's difficult to get a sense
of the prototypes performance with sunny days.
"""

The probability of it being sunny, cloudy, or rainy on day 5 is [0.6608 0.2752 0.064 ], so it will likely still be sunny.

Forward Step for k=1: [0. 0. 1.]
Forward Step for k=2: [0.08 0.42 0.  ]
Forward Step for k=3: [0.0928 0.1288 0.    ]
Forward Step for k=4: [0.      0.      0.02576]
Forward Step for k=5: [0.0030912 0.0046368 0.       ]

Filtered Estimate for k=1: [0. 0. 1.]
Filtered Estimate for k=2: [0.16 0.84 0.  ]
Filtered Estimate for k=3: [0.41877256 0.58122744 0.        ]
Filtered Estimate for k=4: [0. 0. 1.]
Filtered Estimate for k=5: [0.4 0.6 0. ]

Backwards Step for k=5: [1 1 1]
Backwards Step for k=4: [0.56 0.4  0.36]
Backwards Step for k=3: [0.    0.072 0.072]
Backwards Step for k=2: [0.01008 0.02016 0.03024]
Backwards Step for k=1: [0.0052416 0.0068544 0.009072 ]

Smoothed Value for k=1: [0. 0. 1.]
Smoothed Value for k=2: [0.08695652 0.91304348 0.        ]
Smoothed Value for k=3: [0. 1. 0.]
Smoothed Value for k=4: [0. 0. 1.]
Smoothed Value for k=5: [0.4 0.6 0. ]
The pro